# 3次元マルバツゲームを作ろう！

## まずは二次元で作ってみる

In [44]:
player1 = str(input("一人目のプレイヤーの名前を入力してください:"))
print(f"{player1}さんの記号は●です")
player2 = str(input("二人目のプレイヤーの名前を入力してください:"))
print(f"{player2}さんの記号は✖です")

さんの記号は●です
あさんの記号は✖です


In [1]:
# 初期設定
import numpy as np 
X = 0
Y = 0
BC = [0,0,0,0]
board = [] #盤面の設計
for i in range(len(BC)):
    board.append(BC)
board3D =[board,board,board] #3次元に
board3D = np.array(board3D)
board = np.array(board)
class InputMark():
    def __init__(self,x ,y):
        self.x = x
        self.y = y

In [10]:
import numpy as np 
import PySimpleGUI as sg
class InputMark():
    def __init__(self,x,y):
        self.x = x
        self.y = y
        #self.nowPlay = 0

    def WriteError(self):
        print("Error! 1~4の値を代入してください。例：1 3")

    def InputXY(self):
        flag = False
        while flag == False:
            wherePoint = input("半角空白で区切って置く場所を決めてください。順序はx,yです。")
            if wherePoint == "exit":
                flag = True
            if " " not in wherePoint:
                self.WriteError()  #クラス内の関数を呼び出したいならself.関数名とする（その関数の方にも引数selfを入れておくこと）
                continue
            
            self.x = wherePoint.split(" ")[0]
            self.y = wherePoint.split(" ")[1]
            
            if self.x not in ["1","2","3","4"]:
                self.WriteError()
                continue
            if self.y not in ["1","2","3","4"]:
                self.WriteError()
                continue

            self.x = int(self.x)
            self.y = int(self.y)
            
            if self.board[-(self.y -1+1)][self.x -1] in self.oxList: #重複をさせなくする
                print("既にマークが付いています、別の箇所を選択してください")
                continue

            if 0 < self.x  <= 4 and 0 < self.y <= 4:
                flag = True
            else:
                self.WriteError()
                continue
    
    def PlayerSet(self):
        self.player1 = str(input("一人目のプレイヤーの名前を入力してください:"))
        print(f"{self.player1}さんの記号は●です")

        self.player2 = str(input("二人目のプレイヤーの名前を入力してください:"))
        print(f"{self.player2}さんの記号は✖です")

        self.playerList = [self.player1, self.player2]




class App(InputMark):
    #bingo = False bingoはフラグではあるが定数ではないのでinitのなかに入れる（クラス変数の役割ではない）
    #board = [] ここは予想通りいらなかった

    def __init__(self, x=1, y=1):
        super().__init__(x,y)
        self.nowPlay = 0
        self.bingo = False
        self.count = 0
        #self.board = board # ここCreateBoardがやってくれるからいらない。init以外のところでself.定義してもいい
        """
        initでself設定するのはインスタンス定義の時に毎回初期化したい変数だけ（ここに入れておかないと二つ目のインスタンス
        を作った時に一つ目のインスタンスの変数を引き継いでしまう。）
        引数にnowPlayを入れていないが、self.nowPlay = 1とはしている。このことでインスタンス生成時に毎回定義するのは面倒だか         らしたくないが、毎回初期化はしたいという要望を叶えられる。
        """
        
    def CreateBoard(self):
        BC = [0,0,0,0]
        board = [] #盤面の設計
        for i in range(len(BC)): # n*nの盤面を作る
            board.append(BC)
        #self.board3D =[board,board,board] #3次元に
        #self.board3D = np.array(board3D)
        self.board = np.array(board)



    def Switch(self):
        self.oxList = ["●","✖️"]
        print("プレイヤーが切り替わります。")
        print(f"次は{self.playerList[self.nowPlay]}さんの番です")
        if self.nowPlay == 0:
            self.marubatu = self.oxList[0]
            self.nowPlay = 1 #nowPlayをここで変えているが、本当に変わるのはPlotMarkメソッドを実行したあと

        elif self.nowPlay == 1:
            self.marubatu = self.oxList[1]
            self.nowPlay = 0


    def PlotMark(self):
        self.board[-(self.y -1+1)][self.x -1]= 1 # 入力した座標をデカルト座標系と一致させる

        if type(self.board) != list:
            self.board = self.board.tolist()
        for row_num in range(len(self.board)):
            for col_num in range(4):
                if self.board[row_num][col_num] == 0 :
                    self.board[row_num][col_num] = "□"      #空白部分は□で表示
                elif self.board[row_num][col_num] == 1 :
                    self.board[row_num][col_num] = self.marubatu

        for i in self.board:
            print(i)
        print("=========================")


        # 一列揃うとビンゴの表示をしてくれる関数を作ろう
    def RowBingo(self):
            for i in range(4):
                if self.board[i][0] == self.marubatu and self.board[i][1] == self.marubatu \
                and self.board[i][2] == self.marubatu and self.board[i][3] == self.marubatu:
                    print(f"ビンゴ！y={4-i}で揃った！") #0+4=4,i+j=4となるように別の関数で作っておく
                    print(f"決着！　{self.playerList[self.nowPlay -1]}さんの勝利です！")
                    self.bingo =True # 混乱を避けるために、self.bingoではなく、クラス.クラス変数とした方が無難。
    def ColBingo(self):
            for i in range(4):
                if self.board[0][i] == self.marubatu and self.board[1][i] == self.marubatu \
                and self.board[2][i] == self.marubatu and self.board[3][i] == self.marubatu:
                    print(f"ビンゴ！x={i+1}で揃った！")
                    self.bingo =True
                    print(f"決着！　{self.playerList[self.nowPlay -1]}さんの勝利です！")
    def DiagonalBingo(self):
            if self.board[0][0] == self.marubatu and self.board[1][1] == self.marubatu \
            and self.board[2][2] == self.marubatu and self.board[3][3] == self.marubatu:

                print(f"ビンゴ！ y=-xで揃った！")
                self.bingo = True
                print(f"決着！　{self.playerList[self.nowPlay -1]}さんの勝利です！")
            if self.board[3][0] == self.marubatu and self.board[2][1] == self.marubatu \
            and self.board[1][2] == self.marubatu and self.board[0][3] == self.marubatu:
                print(f"ビンゴ！ y=xで揃った！")
                print(f"決着！　{self.playerList[self.nowPlay -1]}さんの勝利です！")

                self.bingo = True 
    
    def CheckDrow(self):
        self.count += 1
        if self.count == len(self.board)* len(self.board[0]):
            print("全てのマスが埋まりました、今回の勝負は引き分けです")
            self.bingo = True
        

In [11]:
def main():
    test = App()
    test.CreateBoard()
    test.PlayerSet()
    print("ゲームスタート！")
    while test.bingo == False:
        test.Switch()
        test.InputXY()
        test.PlotMark()
        test.ColBingo()
        test.DiagonalBingo()
        test.RowBingo()
        test.CheckDrow()

In [12]:
main()

ああああさんの記号は●です
っっっっっっっっdさんの記号は✖です
ゲームスタート！
プレイヤーが切り替わります。
次はああああさんの番です
['□', '□', '□', '□']
['□', '□', '□', '□']
['□', '□', '□', '□']
['●', '□', '□', '□']
プレイヤーが切り替わります。
次はっっっっっっっっdさんの番です
['□', '□', '□', '□']
['□', '□', '□', '□']
['□', '✖️', '□', '□']
['●', '□', '□', '□']
プレイヤーが切り替わります。
次はああああさんの番です
['□', '□', '□', '□']
['□', '□', '●', '□']
['□', '✖️', '□', '□']
['●', '□', '□', '□']
プレイヤーが切り替わります。
次はっっっっっっっっdさんの番です
['□', '□', '□', '□']
['□', '□', '●', '□']
['□', '✖️', '□', '□']
['●', '✖️', '□', '□']
プレイヤーが切り替わります。
次はああああさんの番です
['□', '□', '□', '●']
['□', '□', '●', '□']
['□', '✖️', '□', '□']
['●', '✖️', '□', '□']
プレイヤーが切り替わります。
次はっっっっっっっっdさんの番です
['□', '□', '□', '●']
['□', '✖️', '●', '□']
['□', '✖️', '□', '□']
['●', '✖️', '□', '□']
プレイヤーが切り替わります。
次はああああさんの番です
既にマークが付いています、別の箇所を選択してください
['□', '□', '□', '●']
['□', '✖️', '●', '●']
['□', '✖️', '□', '□']
['●', '✖️', '□', '□']
プレイヤーが切り替わります。
次はっっっっっっっっdさんの番です
['□', '✖️', '□', '●']
['□', '✖️', '●', '●']
['□', '✖️', '□', '□']
['●', '✖️', '□', '□']

In [ ]:
    def InputXY(self):
        flag = False
        while flag == False:
            sg.theme('LightGreen10')
            layout = [
            [sg.Text(f'名前: {self.playerList[self.nowPlay]}', size=(15, 2))],
            [sg.Text('x y :', size=(30, 3)), sg.InputText("半角空白で区切って置く場所を決めてください。順序はx,yです。")],
            [sg.Submit(button_text='実行')]
            ]
            """
            wherePoint = input("半角空白で区切って置く場所を決めてください。順序はx,yです。")
            if wherePoint == "exit":
                flag = True
            if " " not in wherePoint:
                self.WriteError()  #クラス内の関数を呼び出したいならself.関数名とする（その関数の方にも引数selfを入れておくこと）
                continue
            
            self.x = wherePoint.split(" ")[0]
            self.y = wherePoint.split(" ")[1]
            """
            # セクション 2 - ウィンドウの生成
            window = sg.Window('○✖ゲーム', layout)

            event, values = window.read()

            if event is None:
                print('exit')
                break

            if event == '実行':
                self.x = values[0].split(" ")[0]
                self.y= values[0].split(" ")[1]

                # ポップアップ
                #sg.popup()
        # セクション 4 - ウィンドウの破棄と終了
        #window.close()
            if self.x not in ["1","2","3","4"]:
                self.WriteError()
                continue
            if self.y not in ["1","2","3","4"]:
                self.WriteError()
                continue

            self.x = int(self.x)
            self.y = int(self.y)
            
            if self.board[-(self.y -1+1)][self.x -1] in self.oxList: #重複をさせなくする
                print("既にマークが付いています、別の箇所を選択してください")
                continue

            if 0 < self.x  <= 4 and 0 < self.y <= 4:
                flag = True
            else:
                self.WriteError()
                continue

#### 補足: 「インスタンス変数」と「ローカル変数」を区別する。
ちなみに __init__ 関数内で self を記載しなかった変数は、外部から参照できません。 なぜなら __init__ 関数の中だけで参照できる変数として定義されてしまっているからです。 このように関数の外側から見れない変数を「ローカル変数」と呼びます


### クラス変数は class名（インスタンス名）.~と表す。例えば、bingoがクラス変数なので
test = App()

test.bingo = ...のようにかく

def ... (self):　とかいたら他のクラス内のdefと共有できるみたいなイメージだがあってる？

## これからやること：gui実装、3D化、3DverのGUI実装


In [31]:
import PySimpleGUI as sg

#  セクション1 - オプションの設定と標準レイアウト
sg.theme('Dark Blue 3')

layout = [
    [sg.Text('Python GUI')],
    [sg.Text('名前', size=(15, 1)), sg.InputText('○○〇×××')],
    [sg.Text('住所', size=(15, 1)), sg.InputText('△△△△村')],
    [sg.Text('電話番号', size=(15, 1)), sg.InputText('xxx-xxx-xxx')],
    [sg.Submit(button_text='実行ボタン')]
]

# セクション 2 - ウィンドウの生成
window = sg.Window('住所を入力', layout)

# セクション 3 - イベントループ
while True:
    event, values = window.read()

    if event is None:
        print('exit')
        break

    if event == '実行ボタン':
        show_message = "名前：" + values[0] + 'が入力されました。\n'
        show_message += "住所：" + values[1] + 'が入力されました。\n'
        show_message += "電話番号：" + values[2] + "が入力されました。"
        #print(show_message)

        # ポップアップ
        sg.popup(show_message)

# セクション 4 - ウィンドウの破棄と終了
window.close()

exit


In [37]:
import PySimpleGUI as sg

#  セクション1 - オプションの設定と標準レイアウト
sg.theme('LightGreen10')

layout = [
    [sg.Text('Python GUI')],
    [sg.Text('名前', size=(15, 1)), sg.InputText('○○〇×××')],
    [sg.Text('住所', size=(15, 1)), sg.InputText('△△△△村')],
    [sg.Text('電話番号', size=(15, 1)), sg.InputText('xxx-xxx-xxx')],
    [sg.Submit(button_text='実行ボタン')]
]

# セクション 2 - ウィンドウの生成
window = sg.Window('住所を入力', layout)

# セクション 3 - イベントループ
while True:
    event, values = window.read()

    if event is None:
        print('exit')
        break

    if event == '実行ボタン':
        show_message = "名前：" + values[0] + 'が入力されました。\n'
        show_message += "住所：" + values[1] + 'が入力されました。\n'
        show_message += "電話番号：" + values[2] + "が入力されました。"
        #print(show_message)

        # ポップアップ
        sg.popup(show_message)

# セクション 4 - ウィンドウの破棄と終了
window.close()

exit
